## Carga de librerías necesarias para el procesamiento del dataset

In [2]:
# Se importan las librerias a utilizar
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

## Análisis del dataset

### Lectura del archivo de datos

In [9]:
# Se hace la lectura del dataset del escenario y se obtiene su cabecera
academicos = pd.read_csv('../datasets/datos_academicos.csv', ',', index_col='Unnamed: 0')
academicos.head()

,unidad_academica,carrera,nro_inscripcion,regular,cnt_readmisiones,calidad,fecha_ingreso_alumno,anio_plan_estudios
0,FCEQN,170,FCEQN-3342,S,0.0,A,02/28/2011,2005.0
1,FCEQN,102,FCEQN-5396,S,0.0,A,12/12/2010,2005.0
2,FCEQN,143,FCEQN-3162,S,0.0,A,03/07/2011,2008.0
3,FCEQN,106,FCEQN-5739,S,0.0,A,03/12/2011,2008.0
4,FCEQN,172,4683,S,0.0,A,03/15/2011,2005.0


### Registro de meta-datos del dataset

Se van a registrar:
* Los tipos de datos por atributo
* El rango de valores que presenta
* Algunas características de los valores

In [12]:
# Se define una función general para obtener los meta-datos planteados

def descripcionDatosDataset(datos):
    print("Cantidad de filas:", datos.shape[0])
    print("Cantidad de columnas:", datos.shape[1])
    print('-'*100)
    for columna in datos.columns:
        valoresDescripcion = ''
        tipo = ''
        if datos[columna].dtype == 'float64' or datos[columna].dtype == 'int64':
            tipo = 'numérico'
            valoresDescripcion = datos[columna].agg(['min', 'max', 'mean', 'std', 'median'])
        else:
            tipo = 'nominal' #categórico | string | no-numérico
            valoresDescripcion = {'valoresPresentes' : datos[columna].unique(), 
                                  'cantidadNulos' : datos[columna].isna().sum()}
        print('Columna: ' + columna)
        print('Tipo de datos: ' + tipo)
        print('Descripción de valores:')
        if tipo == 'numérico':
            print(valoresDescripcion)
        else:
            print('-- Valores presentes (10 primeros): ' + str(valoresDescripcion['valoresPresentes'][:10]))
            pctNulos = (valoresDescripcion['cantidadNulos'] / datos.shape[0]) * 100
            print('-- Cantidad de nulos: ' + str(valoresDescripcion['cantidadNulos']) + ' = ' + "{0:.2f}".format(pctNulos) + '%')
        print('-'*100)

In [13]:
# Se analiza el dataset
descripcionDatosDataset(academicos)

Cantidad de filas: 2316
Cantidad de columnas: 8
----------------------------------------------------------------------------------------------------
Columna: unidad_academica
Tipo de datos: nominal
Descripción de valores:
-- Valores presentes (10 primeros): ['FCEQN']
-- Cantidad de nulos: 0 = 0.00%
----------------------------------------------------------------------------------------------------
Columna: carrera
Tipo de datos: nominal
Descripción de valores:
-- Valores presentes (10 primeros): ['170' '102' '143' '106' '172' 'SAC' '114' '147' '101' '104']
-- Cantidad de nulos: 2 = 0.09%
----------------------------------------------------------------------------------------------------
Columna: nro_inscripcion
Tipo de datos: nominal
Descripción de valores:
-- Valores presentes (10 primeros): ['FCEQN-3342' 'FCEQN-5396' 'FCEQN-3162' 'FCEQN-5739' '4683' 'FCEQN-5233'
 'FCEQN-5226' 'FCEQN-5117' 'FCEQN-5630' 'FCEQN-5644']
-- Cantidad de nulos: 0 = 0.00%
-------------------------------------

In [16]:
# Vista de los valores únicos en los atributos de tipo string

academicos['regular'].unique()

#Vista de la cantidad de ocurrencias de cada valor

pd.value_counts(academicos['regular'])

S     1908
N      388
D        4
X        3
T        1
SI       1
Name: regular, dtype: int64

In [15]:
# Vista de los valores únicos en los atributos de tipo numérico y obtención de estadísticas


academicos['anio_plan_estudios'].unique()

#Vista de estadísticas del atributo

academicos['anio_plan_estudios'].agg(['min', 'max', 'mean', 'std'])

min     1980.000000
max     2099.000000
mean    2006.703896
std        4.487675
Name: anio_plan_estudios, dtype: float64

### Registro del análisis

1. Evaluación de valores nulos (filas y columnas)
2. Evaluación de formato válido
3. Valores ajustados en rangos (ver anexos)
4. Claves únicas
5. Integridad referencial
6. Cumplimiento de reglas en valores

In [19]:
# Se comienza por obtener la cantidad de filas en las que existen valores nulos
academicos.isnull().sum().sum()

45

In [21]:
# Como segundo paso se pasa a obtener la cantidad de nulos por cada columna
academicos.isna().sum()

# Para poder observar el detalle por filas de las columnas con datos nulos
#academicos[academicos['carrera'].isnull()]

,unidad_academica,carrera,nro_inscripcion,regular,cnt_readmisiones,calidad,fecha_ingreso_alumno,anio_plan_estudios
853,FCEQN,NaN,FCEQN-7254,S,NaN,NaN,04/06/2013,2007.0
854,FCEQN,NaN,FCEQN-6932,S,NaN,NaN,12/12/2012,2011.0


In [22]:
# Verificar formato válido - atributo:nro_inscripcion -

# El formato debería ser FCEQN-#### (el número de matrícula del alumno)
# Se puede usar una evaluación de correspondencia con una expresión regular

academicos['nueva'] = academicos['nro_inscripcion'].astype(str).str.match("\D{5}.\d{1,4}$")

# A partir de esto se podría verificar a través de una nueva columna auxiliar
conteo = academicos[academicos.nueva == False]

# Y se cuenta la cantidad de errores de formateo
conteo.shape[0]

71

In [34]:
# Verificar que los valores de cada atributo se encuentren dentro de los listados anexos

# Atributo: regular
valores_validos = ['S', 'N'] # Se definen los valores validos según el anexo

# Se identifica y cuenta a los valores que no cumplen esa condición (incluye los nulos)
resultado = academicos[academicos.regular.isin(valores_validos) == False] 

# Vista de las tuplas con valores nulos en el atributo
resultado

# Conteo
#resultado.shape[0] 

,unidad_academica,carrera,nro_inscripcion,regular,cnt_readmisiones,calidad,fecha_ingreso_alumno,anio_plan_estudios,nueva
733,FCEQN,106,FCEQN-6979,NaN,0.0,A,12/15/2012,2012.0,True
734,FCEQN,102,FCEQN-7247,NaN,0.0,A,03/30/2013,2011.0,True
788,FCEQN,102,FCEQN-6494,NaN,0.0,A,11/24/2012,2011.0,True
789,FCEQN,106,FCEQN-6735,NaN,0.0,A,12/04/2012,2012.0,True
962,FCEQN,143,FCEQN-6993,NaN,0.0,A,12/15/2012,2008.0,True
963,FCEQN,170,FCEQN-7142,NaN,0.0,A,02/03/2013,2005.0,True
1443,FCEQN,143,FCEQN-7422,NaN,0.0,NaN,12/01/2013,NaN,True
1444,FCEQN,104,FCEQN-7445,NaN,0.0,NaN,12/01/2013,2011.0,True
1472,FCEQN,106,FCEQN-5237,NaN,0.0,P,04/20/2011,2012.0,True
1473,FCEQN,106,FCEQN-6461,NaN,0.0,P,04/14/2012,2012.0,True


In [12]:
#Verificar que los valores de cada atributo se encuentren dentro de los listados anexos

#Atributo: regular
valores = pd.value_counts(academicos['carrera']) #Conteo de ocurrencias por valor (not-null)

academicos[academicos.carrera.isna()] #Para visualizar las tuplas con valores nulos
cantidad_nulos = len(academicos.carrera) - academicos.carrera.count() #Conteo de nulos

valores_validos = ['147',
'172',
'104',
'102',
'101',
'106',
'143',
'SAC',
'170',
'114',
'111',
'105',
'108'] #Se definen los valores validos según el anexo

#Se identifica y cuenta a los valores que no cumplen esa condición (incluye los nulos)
resultado = academicos[academicos.carrera.isin(valores_validos) == False] 
resultado.shape[0]

8

In [13]:
#Atributo: cnt_readmisiones
valores = pd.value_counts(academicos['cnt_readmisiones']) #Conteo de ocurrencias por valor (not-null)

academicos[academicos.cnt_readmisiones.isna()] #Para visualizar las tuplas con valores nulos
cantidad_nulos = len(academicos.cnt_readmisiones) - academicos.cnt_readmisiones.count() #Conteo de nulos

#Se identifica y cuenta a los valores que no cumplen las condiciones vistas
resultado = academicos[academicos.cnt_readmisiones < 0] 
menor_cero = resultado.shape[0]

resultado = academicos[academicos.cnt_readmisiones > 5] 
mayor_cero = resultado.shape[0]

cantidad_nulos += menor_cero + mayor_cero
print(cantidad_nulos)

14


In [14]:
#Atributo: cnt_readmisiones
valores = pd.value_counts(academicos['anio_plan_estudios']) #Conteo de ocurrencias por valor (not-null)

academicos[academicos.anio_plan_estudios.isna()] #Para visualizar las tuplas con valores nulos
cantidad_nulos = len(academicos.anio_plan_estudios) - academicos.anio_plan_estudios.count() #Conteo de nulos

#Se identifica y cuenta a los valores que no cumplen las condiciones vistas
resultado = academicos[academicos.anio_plan_estudios < 2000] 
res = resultado
menor_cero = resultado.shape[0]

resultado = academicos[academicos.anio_plan_estudios > 2019] 
mayor_cero = resultado.shape[0]

cantidad_nulos += menor_cero + mayor_cero #TODO: adaptar nombres variables
print(cantidad_nulos)

resultado

9


,unidad_academica,carrera,nro_inscripcion,regular,cnt_readmisiones,calidad,fecha_ingreso_alumno,anio_plan_estudios,nueva
2244,FCEQN,114,FCEQN-8890,S,0.0,A,05/03/2015,2099.0,True


In [15]:
#Para revisar problemas de claves duplicadas

#Se obtiene el valor de la cantidad de filas actual
cant_antes = len(academicos) 

#Se ordena el dataset según el atributo que se desee evaluar (requerido para el paso siguiente)
academicos.sort_values("nro_inscripcion", inplace=True)

#Se detectan y eliminan los duplicados en un atributo dejando la última ocurrencia
academicos.drop_duplicates(subset ="nro_inscripcion", keep = 'last', inplace = True)

#Se obtiene el valor posterior a la operación
cant_despues = len(academicos)

#Se imprimen ambos valores
print('Antes del análisis de duplicados: ' + str(cant_antes) + ' - Despues del filtrado de duplicados: ' + str(cant_despues))

Antes del análisis de duplicados: 2316 - Despues del filtrado de duplicados: 2316


In [16]:
#Para verificar integridad referencial se tienen que importar los otros datasets

#Lectura del dataset de datos_censales
censales = pd.read_csv('../datasets/datos_censales.csv', ',')
censales.head(5)
eliminar = ['Unnamed: 0']
censales.drop(eliminar, inplace=True, axis=1)
censales.head(5)

#Lectura del dataset de datos_persona
persona = pd.read_csv('../datasets/datos_personas.csv', ',')
persona.head(5)
eliminar = ['Unnamed: 0']
persona.drop(eliminar, inplace=True, axis=1)
persona.head(5)

,unidad_academica,nro_inscripcion,sexo,nacionalidad,fecha_nac_alumno,fecha_egr_sec
0,FCEQN,FCEQN-877,2,1.0,1987,2004
1,FCEQN,FCEQN-1294,2,1.0,1987,2011
2,FCEQN,FCEQN-1351,1,1.0,1987,2011
3,FCEQN,FCEQN-1363,2,1.0,1987,2011
4,FCEQN,FCEQN-1367,2,1.0,1987,2011


In [17]:
#Antes de probar la integración se debe verificar la unicidad de las claves en los otros datasets 
#Se obtiene el valor de la cantidad de filas actual
cant_antes = len(censales) 

#Se ordena el dataset según el atributo que se desee evaluar (requerido para el paso siguiente)
censales.sort_values("insc", inplace=True)

#Se detectan y eliminan los duplicados en un atributo dejando la última ocurrencia
censales.drop_duplicates(subset ="insc", keep = 'last', inplace = True)

#Se obtiene el valor posterior a la operación
cant_despues = len(censales)

#Se imprimen ambos valores
print('Antes: ' + str(cant_antes) + ' - Despues: ' + str(cant_despues))

Antes: 3875 - Despues: 3875


In [18]:
#Antes de probar la integración se debe verificar la unicidad de las claves en los otros datasets 
#Se obtiene el valor de la cantidad de filas actual
cant_antes = len(persona) 

#Se ordena el dataset según el atributo que se desee evaluar (requerido para el paso siguiente)
persona.sort_values("nro_inscripcion", inplace=True)

#Se detectan y eliminan los duplicados en un atributo dejando la última ocurrencia
persona.drop_duplicates(subset ="nro_inscripcion", keep = 'last', inplace = True)

#Se obtiene el valor posterior a la operación
cant_despues = len(persona)

#Se imprimen ambos valores
print('Antes: ' + str(cant_antes) + ' - Despues: ' + str(cant_despues))

Antes: 3875 - Despues: 3875


In [19]:
#Las uniones se hacen de a pares - revisar nombres de atributos
semi_completo = pd.merge(persona, academicos, on='nro_inscripcion', how='inner')
semi_completo.shape[0]

#Para verificar se puede ejecutar
#semi_completo

2316

In [20]:
cambios = {'insc' : 'nro_inscripcion'}
censales.rename(columns=cambios, inplace=True)
censales.head(1)

,ua,nro_inscripcion,estado_civil,sit_lab_alumno,tipo_res_nuevo,sit_lab_padres,estudios_padres
41,FCEQN,1024,1.0,NC,-1.0,-1,-1.0


In [21]:
#Las uniones se hacen de a pares - revisar nombres de atributos
completo = pd.merge(censales, semi_completo, on='nro_inscripcion', how='inner')
completo.shape[0]

#Para verificar se puede ejecutar
completo.head(2)

,ua,nro_inscripcion,estado_civil,sit_lab_alumno,tipo_res_nuevo,sit_lab_padres,estudios_padres,unidad_academica_x,sexo,nacionalidad,fecha_nac_alumno,fecha_egr_sec,unidad_academica_y,carrera,regular,cnt_readmisiones,calidad,fecha_ingreso_alumno,anio_plan_estudios,nueva
0,FCEQN,1024,1.0,NC,-1.0,-1,-1.0,FCEQN,2,1.0,1981,2011,FCEQN,114,S,0.0,A,05/10/2011,2005.0,False
1,FCEQN,1041,1.0,NC,-1.0,-1,-1.0,FCEQN,2,1.0,1979,2011,FCEQN,170,N,0.0,A,03/30/2013,2005.0,False


In [22]:
#Finalmente (última verificación) se revisan cuestiones de aplicación de reglas en los valores del nuevo dataset

#Regla: año egreso secundario no puede ser mayor a año de ingreso a la carrera

#Muestra
fecha_i = completo.fecha_ingreso_alumno.iloc[0]
#fecha_i
ingreso = int(fecha_i[-4:])
#ingreso
egreso = int(completo.fecha_egr_sec.iloc[0])
diferencia = ingreso - egreso
#egreso

print('Ingreso carrera: ' + str(ingreso) + 
      ' Egreso nivel medio: ' + str(egreso) + 
      ' Diferencia: ' + str(diferencia))


Ingreso carrera: 2011 Egreso nivel medio: 2011 Diferencia: 0


In [23]:
#Se puede definir una función para aplicar los cálculos
def reglaAnioSecundario(row):
    #fecha_ingreso_alumno es tipo date
    anio_ingreso = int(row['fecha_ingreso_alumno'][-4:]) #Se aisla el año
    
    anio_egreso_secundaria = int(row['fecha_egr_sec'])
    diferencia = anio_ingreso - anio_egreso_secundaria
    if (diferencia < 0):
        #Es un error ya que no podría entrar al nivel superior sin haber finalizado en el nivel medio
        return 'err'
    else:
        return 'ok'


pd.value_counts(completo.fecha_ingreso_alumno)
completo['fecha_ingreso_alumno'].fillna('12/12/2010',inplace=True)
    

#Se aplica la función para todos los elementos del dataset
completo['diferencia_nivel'] = completo.apply(lambda row: reglaAnioSecundario(row), axis=1)

#Se verifica la cantidad de elementos
aux = completo[completo.diferencia_nivel == 'err']
#len(aux)
aux

,ua,nro_inscripcion,estado_civil,sit_lab_alumno,tipo_res_nuevo,sit_lab_padres,estudios_padres,unidad_academica_x,sexo,nacionalidad,...,fecha_egr_sec,unidad_academica_y,carrera,regular,cnt_readmisiones,calidad,fecha_ingreso_alumno,anio_plan_estudios,nueva,diferencia_nivel
2,FCEQN,1057,1.0,NC,4.0,1,4.0,FCEQN,2,1.0,...,2011,FCEQN,104,S,1.0,A,02/19/2003,2011.0,False,err
4,FCEQN,1382,1.0,NC,1.0,-1,-1.0,FCEQN,2,1.0,...,2011,FCEQN,102,S,2.0,A,02/10/1998,2005.0,False,err
5,FCEQN,1439,1.0,NC,-1.0,-1,-1.0,FCEQN,2,1.0,...,2011,FCEQN,104,N,1.0,P,04/26/2004,2011.0,False,err
7,FCEQN,1488,1.0,NC,-1.0,-1,-1.0,FCEQN,2,1.0,...,2011,FCEQN,105,S,0.0,P,10/08/2008,2008.0,False,err
11,FCEQN,1576,1.0,NC,-1.0,-1,-1.0,FCEQN,1,1.0,...,2011,FCEQN,102,S,2.0,A,07/28/2001,2011.0,False,err
12,FCEQN,1666,1.0,NC,-1.0,-1,-1.0,FCEQN,2,1.0,...,2011,FCEQN,102,S,0.0,A,02/14/1998,2005.0,False,err
15,FCEQN,1711,1.0,NC,-1.0,-1,-1.0,FCEQN,1,1.0,...,2011,FCEQN,102,S,0.0,P,02/19/2003,2011.0,False,err
17,FCEQN,1764,1.0,NC,-1.0,-1,-1.0,FCEQN,1,1.0,...,2011,FCEQN,102,S,0.0,A,02/19/2003,2005.0,False,err
18,FCEQN,1767,1.0,NC,4.0,-1,-1.0,FCEQN,2,1.0,...,2011,FCEQN,102,S,0.0,A,02/19/2003,2005.0,False,err
19,FCEQN,1772,1.0,NC,-1.0,-1,-1.0,FCEQN,2,1.0,...,2011,FCEQN,102,S,1.0,A,02/19/2003,2005.0,False,err


In [26]:
new = completo[completo['estado_civil'] > 1]
new.head()

,ua,nro_inscripcion,estado_civil,sit_lab_alumno,tipo_res_nuevo,sit_lab_padres,estudios_padres,unidad_academica_x,sexo,nacionalidad,...,fecha_egr_sec,unidad_academica_y,carrera,regular,cnt_readmisiones,calidad,fecha_ingreso_alumno,anio_plan_estudios,nueva,diferencia_nivel
9,FCEQN,1554,2.0,NC,1.0,4,4.0,FCEQN,2,1.0,...,2011,FCEQN,104,S,0.0,A,02/28/2015,2011.0,False,ok
16,FCEQN,1736,2.0,No,1.0,1,6.0,FCEQN,2,1.0,...,2011,FCEQN,104,S,0.0,A,02/04/2015,2011.0,False,ok
22,FCEQN,2010,2.0,NC,1.0,-1,-1.0,FCEQN,2,1.0,...,2011,FCEQN,105,S,0.0,A,04/04/2015,2008.0,False,ok
23,FCEQN,2920,2.0,NC,-1.0,-1,-1.0,FCEQN,2,1.0,...,2011,FCEQN,114,S,0.0,A,05/10/2011,2005.0,False,ok
24,FCEQN,309,2.0,NC,1.0,1,3.0,FCEQN,2,1.0,...,2011,FCEQN,104,S,0.0,A,03/02/2011,2005.0,False,ok
